In [1]:
import pymongo
import pandas as pd
import numpy as np
import datetime

In [2]:
# Stupid but smart
def getCorrespondingStartDate(date):
    month = date.month
    year = date.year
    if month - 10 >= 0: 
        return datetime.date(year, 10, 1)
    if month - 7 >= 0:
        return datetime.date(year, 7, 1)
    if month - 4 >= 0:
        return datetime.date(year, 4, 1)
    return datetime.date(year, 1, 1)

In [9]:
# here's my plan. i iterate through features separately. so i can sort by $exists = True.
# helps avoid those annoying key errors, and the mathematical badness of setting nan = 0
# when you're trying to calculate an average. 
# it's NOT efficient, but we're only doing it once. right??
def getDataAnomQuarterly(start_date, end_date):
    client = pymongo.MongoClient('localhost', 27017, maxPoolSize=100)
    db = client['Stocks']
    stocks = ["AAPL US Equity", "WWW US Equity"]

    fields = ['LAST_PRICE', 'EBITDA', 'HISTORICAL_MARKET_CAP', 'LOG_RETURN']
    cols = ['Stock'] + fields
    
    df = pd.DataFrame([], columns=cols)
    
    df_arr = [None for _ in range(len(stocks))]
    
    curr_date = None
    curr_total = 0
    curr_count = 0
    
    for i in range(len(stocks)): 
        stock = stocks[i]
        df = pd.DataFrame([], columns=cols)
        for field in fields:
            curr_stock = db[stock]
            for thing in curr_stock.find({field:{'$exists': True}}).sort([('Date', 1)]):
                date = thing['Date']
                corr_date = getCorrespondingStartDate(date)

                if curr_date is None:
                    curr_count += 1
                    curr_date = corr_date
                    curr_total = thing[field]
                elif (corr_date == curr_date):
                    curr_count += 1
                    curr_total += thing[field]   
                # If the dates don't match, we save the info before moving on. 
                else:
                    df.at[curr_date, field] = curr_total/curr_count
                    df.at[curr_date, 'Stock'] = stock
                    curr_date = corr_date
                    curr_count = 1
                    curr_total = thing[field]

        df_arr[i] = df
        
    print(pd.concat(df_arr).sort_index())
    
    return df_arr


In [10]:
start_date = datetime.datetime(1990, 1, 1)
end_date = datetime.datetime(2019, 1, 1)

dataDict = getDataAnomQuarterly(start_date, end_date)

                     Stock LAST_PRICE   EBITDA HISTORICAL_MARKET_CAP  \
1990-01-01  AAPL US Equity     1.2887  243.463               4961.62   
1990-01-01   WWW US Equity   0.746233      NaN                   NaN   
1990-04-01   WWW US Equity   0.754467      NaN                   NaN   
1990-04-01  AAPL US Equity    1.49257  232.099               5355.14   
1990-07-01  AAPL US Equity     1.2857  201.299               3345.41   
1990-07-01   WWW US Equity   0.614533      NaN                   NaN   
1990-10-01  AAPL US Equity    1.31547  275.523                4915.5   
1990-10-01   WWW US Equity   0.496567      NaN                   NaN   
1991-01-01  AAPL US Equity    2.15177  232.443               8089.01   
1991-01-01   WWW US Equity     0.6008    2.541               59.5832   
1991-04-01   WWW US Equity   0.633733    2.206               71.2918   
1991-04-01  AAPL US Equity    1.70833  188.636               4875.83   
1991-07-01   WWW US Equity     0.7901    5.256               76.

In [5]:
print(dataDict)

[                     Stock LAST_PRICE   EBITDA HISTORICAL_MARKET_CAP  \
1990-01-01  AAPL US Equity     1.2887  243.463               4961.62   
1990-04-01  AAPL US Equity    1.49257  232.099               5355.14   
1990-07-01  AAPL US Equity     1.2857  201.299               3345.41   
1990-10-01  AAPL US Equity    1.31547  275.523                4915.5   
1991-01-01  AAPL US Equity    2.15177  232.443               8089.01   
1991-04-01  AAPL US Equity    1.70833  188.636               4875.83   
1991-07-01  AAPL US Equity    1.77087  179.223               5800.91   
1991-10-01  AAPL US Equity     1.8899  312.415               6516.95   
1992-01-01  AAPL US Equity    2.26787  260.673                  7330   
1992-04-01  AAPL US Equity     1.9985  258.188               5395.66   
1992-07-01  AAPL US Equity    1.64137  191.714               5390.79   
1992-10-01  AAPL US Equity    2.02083  280.186               6952.62   
1993-01-01  AAPL US Equity     1.9524  211.908               62

In [6]:
s_len = 5
f_len = 10

a = np.empty((s_len, f_len))

In [7]:
a

array([[2.22803071e-314, 2.22759205e-314, 2.22759031e-314,
        2.22759160e-314, 2.22759133e-314, 2.22759140e-314,
        2.22759146e-314, 2.22803052e-314, 2.22803043e-314,
        2.22803100e-314],
       [2.22803062e-314, 2.22803033e-314, 2.22803027e-314,
        2.22803021e-314, 2.22803014e-314, 2.22803008e-314,
        2.22803002e-314, 2.22802995e-314, 2.22802989e-314,
        2.22802983e-314],
       [2.22802976e-314, 2.22802970e-314, 2.22802964e-314,
        2.22802957e-314, 2.22802951e-314, 2.22802934e-314,
        2.22802945e-314, 2.22802939e-314, 2.22802929e-314,
        2.22802926e-314],
       [2.22803114e-314, 2.22803087e-314, 2.22803119e-314,
        2.22803093e-314, 2.22803081e-314, 2.22803059e-314,
        2.22802942e-314, 2.22814665e-314, 2.22814684e-314,
        2.22814687e-314],
       [2.22814690e-314, 2.22814693e-314, 2.22814696e-314,
        2.22814700e-314, 2.22814703e-314, 2.22814706e-314,
        2.22814709e-314, 2.22814712e-314, 2.22814715e-314,
        2.2

In [8]:
print(a[4])

[2.22814690e-314 2.22814693e-314 2.22814696e-314 2.22814700e-314
 2.22814703e-314 2.22814706e-314 2.22814709e-314 2.22814712e-314
 2.22814715e-314 2.22814719e-314]
